## Excel Sheet Concatenater
Concatenating excel files under the same folder, with the freedom of:
- concatenating into different files based on **user-defined grouping** (such as file styles/formats)
- **cherry-picking** the files/sheets required and ignore the irrelevant ones - based on user input!
- indexing your data to differentiate them by source file

In [1]:
import os
import glob
import xlrd
import pandas as pd
from openpyxl import load_workbook

Step 1: Input the path to your files that you want to combine

In [2]:
os.chdir(r'to_combine')
extension1 = 'xls'
extension2 = 'xlsx'
extension3 = 'xlsm'
All_f = [i for i in glob.glob('*.{}'.format(extension1))]
All_f.extend([i for i in glob.glob('*.{}'.format(extension2))])
All_f.extend([i for i in glob.glob('*.{}'.format(extension3))])

In [3]:
# In this example, theres 3 files under the path
All_f

['Class_A.xlsx', 'Class_B.xlsx', 'Class_C.xlsx']

Next we will retrieve all the sheet names from the 3 files in All_f, and generate the reference dataframe for our mapping purposes, in which:
- the same "file style" will go into the same excel file
- the same "dtype" will go into the same tab (not necessarily the same file)

This is also where you can get a sneak peek on all the sheets/tabs for each excel file under the same folder

In [4]:
# Get tab names AND exporting reference dataframe (for tabs mapping)
df_tabs = pd.DataFrame()
for f in All_f:
    try:
        xls = xlrd.open_workbook(f, on_demand=True)
        df_tabs[f] = [xls.sheet_names()]
    except:
        df_tabs[f] = [load_workbook(f, read_only=True, keep_links=False).sheetnames]
df_tabs = df_tabs.transpose()
df_tabs = df_tabs.reset_index().rename(columns={'index':'file',0:'tabs'}).explode('tabs')
df_tabs['file style']=pd.Series([])
df_tabs['dtype']=pd.Series([])
df_tabs

C:\Users\RJ589AH\AppData\Local\Temp\ipykernel_10904\608237739.py:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_tabs['file style']=pd.Series([])
C:\Users\RJ589AH\AppData\Local\Temp\ipykernel_10904\608237739.py:12: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_tabs['dtype']=pd.Series([])


,file,tabs,file style,dtype
0,Class_A.xlsx,Junior,NaN,NaN
0,Class_A.xlsx,Mid,NaN,NaN
0,Class_A.xlsx,High,NaN,NaN
0,Class_A.xlsx,other stuffs,NaN,NaN
1,Class_B.xlsx,Junior,NaN,NaN
1,Class_B.xlsx,Mid,NaN,NaN
1,Class_B.xlsx,High,NaN,NaN
1,Class_B.xlsx,other stuffs,NaN,NaN
2,Class_C.xlsx,Junior,NaN,NaN
2,Class_C.xlsx,Mid,NaN,NaN


Above shows the reference dataframe for manual external input (i.e. grouping your required tabs), our target is to:
- concatenate the tabs named "Junior" and "High" in one file, but different tabs
- concatenate "other stuffs" in another file
- ignore all the "Mid" tabs

In this example, I will export it to a csv and do that outside

In [5]:
df_tabs.to_csv('_dfref.csv', index=False, encoding='utf-8')

Now I will read in the tab mapping file back into python

In [6]:
_dfref = pd.read_csv('_dfref.csv')
_dfref = _dfref.dropna().reset_index().drop('index', axis=1)
print("Data type:", _dfref.dtype.unique())

grouped = _dfref.groupby('file style')
dfrefs = [group.reset_index().drop('index',axis=1) for name, group in grouped]
print("File styles:", [name for name,group in grouped])

Data type: ['Junior' 'High' 'other']
File styles: ['Style_1', 'Style_2']


In [7]:
_dfref

,file,tabs,file style,dtype
0,Class_A.xlsx,Junior,Style_1,Junior
1,Class_A.xlsx,High,Style_1,High
2,Class_A.xlsx,other stuffs,Style_2,other
3,Class_B.xlsx,Junior,Style_1,Junior
4,Class_B.xlsx,High,Style_1,High
5,Class_B.xlsx,other stuffs,Style_2,other
6,Class_C.xlsx,Junior,Style_1,Junior
7,Class_C.xlsx,High,Style_1,High
8,Class_C.xlsx,other stuffs,Style_2,other


From above input sample for the mapping file, you can see that I have removed all the "Mid" tabs, as they are unwanted.

Then based on the input we concatenate the sheets, and output the results in Excel files

In [8]:
for fStyle in dfrefs:
    cat_Junior = pd.DataFrame()
    cat_High = pd.DataFrame()
    cat_Other = pd.DataFrame()
    
    for idx,name in enumerate(fStyle.index): # for each excel style
        tab = fStyle.loc[idx]
        match tab['dtype']:
            case 'Junior':
                df = pd.read_excel(tab['file'], sheet_name=tab['tabs'])
                df.insert(loc=0, column='file_name', value=tab['file'])
                df.insert(loc=1, column='tab_name', value=tab['tabs'])

                cat_Junior = pd.concat([cat_Junior, df], ignore_index=True)
            case 'High':
                df = pd.read_excel(tab['file'], sheet_name=tab['tabs'])
                df.insert(loc=0, column='file_name', value=tab['file'])
                df.insert(loc=1, column='tab_name', value=tab['tabs'])

                cat_High = pd.concat([cat_High, df], ignore_index=True)
            case 'other':
                df = pd.read_excel(tab['file'], sheet_name=tab['tabs'])
                df.insert(loc=0, column='file_name', value=tab['file'])
                df.insert(loc=1, column='tab_name', value=tab['tabs'])

                cat_Other = pd.concat([cat_Other, df], ignore_index=True)

    with pd.ExcelWriter('_style_' + fStyle['file style'].iloc[0] + '.xlsx') as writer:
        cat_Junior.to_excel(writer, sheet_name='cat_Junior', index=False)
        cat_High.to_excel(writer, sheet_name='cat_High', index=False)
        cat_Other.to_excel(writer, sheet_name='cat_Other', index=False)

You might notice that the entire "match" function seems unnecessary as all the subsequent treatments are actually the same, indeed you may remove the match function if "dtype" is not important to you.

However, this leave some flexibility in case you are having a dataset which you would like to skip different number of rows or perform different kind of data cleansing for different "dtype". This way, you can easily insert your data cleansing function in between.

Below shows some sample output...

In [9]:
pd.read_excel('_style_Style_1.xlsx', sheet_name='cat_Junior')

,file_name,tab_name,Class,Age,Weight
0,Class_A.xlsx,Junior,Junior,3,30
1,Class_A.xlsx,Junior,Junior,4,32
2,Class_A.xlsx,Junior,Junior,5,31
3,Class_B.xlsx,Junior,Junior,3,30
4,Class_B.xlsx,Junior,Junior,4,32
5,Class_B.xlsx,Junior,Junior,5,31
6,Class_C.xlsx,Junior,Junior,3,30
7,Class_C.xlsx,Junior,Junior,4,32
8,Class_C.xlsx,Junior,Junior,5,31


In [10]:
pd.read_excel('_style_Style_2.xlsx', sheet_name='cat_Other')

,file_name,tab_name,Col1,Col2,Col3,Col4,Name
0,Class_A.xlsx,other stuffs,5.1,3.5,1.4,0.2,Iris-setosa
1,Class_A.xlsx,other stuffs,4.9,3.0,1.4,0.2,Iris-setosa
2,Class_A.xlsx,other stuffs,4.7,3.2,1.3,0.2,Iris-setosa
3,Class_A.xlsx,other stuffs,4.6,3.1,1.5,0.2,Iris-setosa
4,Class_A.xlsx,other stuffs,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...,...
82,Class_C.xlsx,other stuffs,4.8,3.4,1.9,0.2,Iris-setosa
83,Class_C.xlsx,other stuffs,5.0,3.0,1.6,0.2,Iris-setosa
84,Class_C.xlsx,other stuffs,5.0,3.4,1.6,0.4,Iris-setosa
85,Class_C.xlsx,other stuffs,5.2,3.5,1.5,0.2,Iris-setosa
